### Mouse Protein dataset regression (accuracy)

The 'Data_Cortex_Nuclear.csv' dataset is from the **abidlabs/contrastive** Github repository: https://github.com/abidlabs/contrastive/blob/master/experiments/datasets/Data_Cortex_Nuclear.csv

In [ ]:
%pip install contrastive-inverse-regression

In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.tree import DecisionTreeClassifier
import contrastive_inverse_regression
from contrastive_inverse_regression import CIR


data = pd.read_csv('datasets/Data_Cortex_Nuclear.csv')

# foreground data
fg = data.dropna()

# foreground label
Y = fg['class']
Y = pd.Categorical(Y)
Y = Y.rename_categories({'c-CS-m': '0', 'c-CS-s': '1', 'c-SC-m': '2', 'c-SC-s': '3',
                         't-CS-m': '4', 't-CS-s': '5', 't-SC-m': '6', 't-SC-s': '7'})
Y = Y.astype(float)
labels = np.unique(Y)      # set of unique foreground labels

# foreground slices
L = len(labels)         # number of foreground classes/slices
fg = fg.iloc[:, 1:78]
n = fg.shape[0]  # foreground sample size
X = fg - np.mean(fg, axis=0)
X = X.values

# background data
bg = data[data['Genotype'] == 'Control'].copy()
bg = bg.dropna()

# background label
Yt = bg['Behavior']
Yt = pd.Categorical(Yt)
Yt = Yt.rename_categories({'C/S': '0', 'S/C': '1'})
Yt = Yt.astype(float)
labelst = np.unique(Yt)   # set of unique background labels
Lt = len(labelst)       # number of background classes/slices
bg = bg.iloc[:, 1:78]
m, p = bg.shape

# tuning parameter
alpha = 0.0002
d = 2

print("CIR......")
accurary_group = []


for i in range(1):
    V_CIR = CIR(fg, Y, bg, Yt, alpha, d)
    X_CIR = X @ V_CIR
    mdl_CIR_KNN = KNeighborsClassifier(n_neighbors=1)
    # kf = KFold(n_splits=10)
    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cross_val_results = cross_val_score(
        mdl_CIR_KNN, X_CIR, Y, cv=10, scoring='accuracy')
    # print(cross_val_results)
    Accuracy_CIR_KNN = np.mean(cross_val_results)
    accurary_group.append(Accuracy_CIR_KNN)

print("this is CIR_KNN ", accurary_group)
print("This is mean ", np.mean(accurary_group))
print("This is standard deviation", np.std(accurary_group))


accurary_group_tree = []

for i in range(1):
    V_CIR = CIR(fg, Y, bg, Yt, alpha, d)
    X_CIR = X @ V_CIR
    mdl_CIR_Tree = DecisionTreeClassifier()
    # kf = KFold(n_splits=10)
    # kf = KFold(n_splits=10, shuffle=True, random_state=42)
    cross_val_results = cross_val_score(
        mdl_CIR_Tree, X_CIR, Y, cv=10, scoring='accuracy')
    # print(cross_val_results)
    Accuracy_CIR_Tree = cross_val_results.mean()
    accurary_group_tree.append(Accuracy_CIR_Tree)

print("this is CIR_Tree ", accurary_group_tree)
print("This is mean ", np.mean(accurary_group_tree))
print("This is standard deviation ", np.std(accurary_group_tree))


CIR......


/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


---------------------------------------------------

Results for Scaled Gradient Projection Method 

---------------------------------------------------

   Obj. function = 2.411595e+00

   Gradient norm = 9.463887e-04 

   ||X^T*X-I||_F = 2.49e-16

   Iteration number = 3000

   Cpu time (secs) = 52.1963

   Number of evaluation(Obj. func) = 3238

this is CIR_KNN  [0.7497727272727273]
This is mean  0.7497727272727273
This is standard deviation 0.0


/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/lianama/Library/Python/3.10/lib/python/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(


---------------------------------------------------

Results for Scaled Gradient Projection Method 

---------------------------------------------------

   Obj. function = 2.411540e+00

   Gradient norm = 1.858041e-04 

   ||X^T*X-I||_F = 2.25e-16

   Iteration number = 3000

   Cpu time (secs) = 4.8057

   Number of evaluation(Obj. func) = 3354

this is CIR_Tree  [0.7099025974025975]
This is mean  0.7099025974025975
This is standard deviation  0.0
